# Querying by object

It is possible to simply run a query with an object from our storage that results in the most relevant items to the said object.

In [1]:
%pip install superlinked==8.12.2

In [2]:
import pandas as pd

from superlinked.framework.common.schema.id_schema_object import IdField
from superlinked.framework.common.schema.schema import schema
from superlinked.framework.common.schema.schema_object import String
from superlinked.framework.dsl.index.index import Index
from superlinked.framework.dsl.space.text_similarity_space import TextSimilaritySpace

from superlinked.framework.dsl.executor.in_memory.in_memory_executor import (
    InMemoryExecutor,
)
from superlinked.framework.dsl.source.in_memory_source import InMemorySource
from superlinked.framework.dsl.query.query import Query

pd.set_option("display.max_colwidth", 100)

In [3]:
@schema
class Paragraph:
    id: IdField
    body: String


paragraph = Paragraph()

body_space = TextSimilaritySpace(
    text=paragraph.body, model="sentence-transformers/all-mpnet-base-v2"
)
paragraph_index = Index(body_space)

Now let's add some data to our space and fire up a running executor

In [4]:
source: InMemorySource = InMemorySource(paragraph)
executor = InMemoryExecutor(sources=[source], indices=[paragraph_index])
app = executor.run()

In [5]:
source.put(
    [
        {"id": "paragraph-1", "body": "Glorious animals live in the wilderness."},
        {
            "id": "paragraph-2",
            "body": "Growing computation power enables advancements in AI.",
        },
        {
            "id": "paragraph-3",
            "body": "The flora and fauna of a specific habitat highly depend on the weather.",
        },
    ]
)

## Using the .with_vector clause

Provides the opportunity to search with the vector of an object in our database. This is useful for example for recommending items for a user based on it's vector.

In [6]:
query = Query(paragraph_index).find(paragraph).with_vector(paragraph, "paragraph-3")

In [7]:
result = app.query(query)

result.to_pandas()

,body,id
0,The flora and fauna of a specific habitat highly depend on the weather.,paragraph-3
1,Glorious animals live in the wilderness.,paragraph-1
2,Growing computation power enables advancements in AI.,paragraph-2


The first result is the one we are searching with, the second is the more related one, and finally the less connected paragraph body comes.